# Juego de disparejo

Considere un juego de disparejo entre cuatro jugadores, en el que cada jugador adopta la siguiente estrategia:
- El jugador 1 siempre escoge al azar entre las dos opciones.
- El jugador 2 escoge siempre hacia abajo sin importar lo que haya ocurrido anteriormente.
- El jugador 3 escoge la última opción ganadora de las partidas anteriores. En la primera jugada, escoge arriba.
- El jugador 4 escoge aquello opuesto al jugador 1 en la última partida. En la primera jugada, escoge al azar.
Utilice ```AgentPy``` para generar una simulación del juego, y simule 1000 juegos consecutivos. Muestre los resultados obtenidos para cada uno de los jugadores al final de la simulación.




In [1]:
import agentpy as ap
import numpy as np
from collections import Counter

**Definir un objeto donde se va a almacenar la información del juego**

In [2]:
HISTORY = {
    "last_win": None,
    "plays": [],
    "winners": []
}

**Definir la clase del jugador, donde se tendrán las reglas que debe de seguir cada uno de ellos**

In [3]:
class Player(ap.Agent):
    
    def setup(self):
        """
        Método que configura al agente
        """
        self.id = 0
        self.last_play = 0
        
    def play(self):
        """
        Método que define la acción a tomar
            1 -> arriba
            2 -> abajo
        """
        global HISTORY
        
        # Escoge un número aleatorio entre 1 y 2
        if self.id == 1:
            self.last_play = np.random.randint(0,2) + 1
        
        # Siempre escoge 2 (abajo)
        elif self.id == 2:
            self.last_play = 2
            
        # Siempre escogerá última opción que ganó, en caso de que no haya ganado ninguna todavía entonces escogerá 1
        elif self.id == 3:
            self.last_play = HISTORY["last_win"] if HISTORY["last_win"] else 1
        
        # Siempre escogerá la opción contraria a la que escojió el jugador 1 en la partida anterior. Al principio es al azar
        elif self.id == 4:
            if len(HISTORY["plays"]) == 0:
                self.last_play = np.random.randint(0,2) + 1
            elif HISTORY["plays"][-1][0] == 1:
                self.last_play = 2
            elif HISTORY["plays"][-1][0] == 2:
                self.last_play = 1
            

**Definir la información del juego, donde se va a inicializar cada uno de los jugadores además de definir como se va a hacer cada una de las jugadas**

In [4]:
class Game(ap.Model):
    
    def setup(self):
        """
        Configuración inicial del modelo (sobrecarga)
        """
        self.agents = ap.AgentList(self, self.p.agents, Player)
        
        # A cada uno de los agentes asignarles un id diferente
        self.agents.id = ap.AttrIter([1, 2, 3, 4])
        
        
    def step(self):
        """
        Define el comportamiento del sistema paso a paso (sobrecarga)
        """
        global HISTORY
        
        # Llamar al método play de cada agente
        self.agents.play()
        
        # Comparar los resultados y definir al ganador
        results = [agent.last_play for agent in self.agents]
        HISTORY["plays"].append(results)
        
        # Definir si hay un ganador
        results = np.array(results) - 1
        if np.count_nonzero(results) == 1:
            HISTORY["last_win"] = 2
            for i in range(4):
                if HISTORY["plays"][-1][i] == 2:
                    HISTORY["winners"].append(i + 1)
                    break
        elif np.count_nonzero(results) == 3:
            HISTORY["last_win"] = 1
            for i in range(4):
                if HISTORY["plays"][-1][i] == 1:
                    HISTORY["winners"].append(i + 1)
                    break


**Definir los parámetros del juego**

In [5]:
parameters = {
    'agents': 4,
    'steps': 1000,
    'seed': 13
}

**Correr la simulación del juego**

In [6]:
model = Game(parameters)
model.run()

Completed: 1000 steps
Run time: 0:00:00.142021
Simulation finished


DataDict {
'info': Dictionary with 9 keys
'parameters': 
    'constants': Dictionary with 3 keys
'reporters': DataFrame with 1 variable and 1 row
}

**Leaderboard**

In [7]:
ordered_players = Counter(HISTORY["winners"]).most_common()
wins = 0
for index, result in enumerate(ordered_players):
    print(f"#{index + 1}. Player 0{result[0]} (winner {result[1]} times)")
    wins += result[1]
    
print(f"-> {parameters['steps'] - wins} times were played without a winner")

#1. Player 02 (winner 190 times)
#2. Player 01 (winner 137 times)
#3. Player 03 (winner 136 times)
#4. Player 04 (winner 53 times)
-> 484 times were played without a winner


**Ver 20 de los juegos que se hicieron para confirmar que se respeten las reglas del juego**

In [8]:
HISTORY["plays"][:20]

[[1, 2, 1, 1],
 [1, 2, 2, 2],
 [1, 2, 1, 2],
 [2, 2, 1, 2],
 [2, 2, 1, 1],
 [2, 2, 1, 1],
 [1, 2, 1, 1],
 [1, 2, 2, 2],
 [2, 2, 1, 2],
 [1, 2, 1, 1],
 [2, 2, 2, 2],
 [2, 2, 2, 1],
 [1, 2, 1, 1],
 [1, 2, 2, 2],
 [1, 2, 1, 2],
 [1, 2, 1, 2],
 [1, 2, 1, 2],
 [2, 2, 1, 2],
 [2, 2, 1, 1],
 [1, 2, 1, 1]]

## Preguntas

**¿Qué jugador obtuvo mejores resultados?**
- Los mejores resultados, por un amplio margen, los obtuvo el jugador 02. Este jugador siempre tomaba la misma posición sin importar los resultados anteriores o lo que pusieran los demás jugadores. Esto lo podemos ver en la penúltima celda, donde vemos las veces que ganó cada uno de los jugadores de las 1000 veces que se realizó el juego.

**¿Alguna de las estrategias fue mejor que las otras?**
- La mejor estrategia fue claramente siempre poner la mano para abajo (la misma posición en todos los juegos). Repetimos el experimento más de 5 veces y siempre el jugador 02 fue el vencedor, además de que por un amplio margen.